In [7]:
import urllib2
import json

API_KEY = 'vjfSeIozbYBgOrnA9HJBuMRQF6arH8I2'
LEFT_BOTTOM= ['-71.089195', '42.342926']
RIGHT_TOP = ['-71.066139', '42.358044']

In [15]:
def get_box_hotel(left_bottom, right_top):
    """Get hotel list according to their geo location(bounded box)
    
    left_bottom and right_top is a length 2 list 
    recording the longitude and lattitude of a bounded rectangular box.
    """
    url = ''.join(('http://terminal2.expedia.com:80/x/geo/features?bbox=',
                   left_bottom[0], '%2C', left_bottom[1], '%2C', right_top[0], 
                   '%2C', right_top[1], '&type=hotel&apikey=', API_KEY))
    data = json.load(urllib2.urlopen(url))
    
    return data

In [16]:
data = get_box_hotel(LEFT_BOTTOM, RIGHT_TOP)
print len(data)

42
